In [ ]:
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Configuración inicial
image_size = (150, 150)  # Tamaño de las imágenes para redimensionar
batch_size = 32

# Ruta al dataset (ajusta esto a tu entorno)
dataset_path = "./COVID-19 Radiography Database"

# Preparación de los datos
categories = ["NORMAL", "COVID", "LUNG_OPACITY", "VIRAL_PNEUMONIA"]
data = []
labels = []

for idx, category in enumerate(categories):
    folder_path = os.path.join(dataset_path, category)
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        try:
            img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
            img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
            data.append(img_array)
            labels.append(idx)
        except Exception as e:
            print(f"Error al procesar {img_path}: {e}")

# Conversión a arrays de NumPy
data = np.array(data)
labels = np.array(labels)

# Dividir en entrenamiento y prueba
train_images, test_images, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42, stratify=labels)

# Dividir conjunto de validación
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels)

# Aumento de datos
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator()

# Crear generadores
train_generator = train_datagen.flow(train_images, train_labels, batch_size=batch_size)
val_generator = val_datagen.flow(val_images, val_labels, batch_size=batch_size)

# Crear el modelo
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(categories), activation='softmax')
])

# Compilación del modelo
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Configurar Early Stopping
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

# Entrenamiento
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping]
)

# Evaluar en el conjunto de prueba
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f"Exactitud en prueba: {test_acc:.2f}")

# Graficar curvas de entrenamiento y validación
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Exactitud entrenamiento')
plt.plot(history.history['val_accuracy'], label='Exactitud validación')
plt.xlabel('Épocas')
plt.ylabel('Exactitud')
plt.legend()
plt.title('Curvas de entrenamiento y validación')
plt.show()

# Predicciones
y_pred = model.predict(test_images)
y_pred_classes = np.argmax(y_pred, axis=1)

# Matriz de confusión
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(test_labels, y_pred_classes)
print("\nMatriz de confusión:\n", cm)

# Informe de clasificación
print("\nInforme de clasificación:\n", classification_report(test_labels, y_pred_classes, target_names=categories))
